In [2]:
import sys
import torch
print("Python version:", sys.version)
print("Torch version:", torch.__version__)
print("CUDA available?", torch.cuda.is_available())
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

Python version: 3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:41:22) [MSC v.1929 64 bit (AMD64)]
Torch version: 2.5.1
CUDA available? True


In [3]:
torch.cuda.empty_cache()

In [4]:
# Adjust filenames as needed
TRAIN_PATH = "UNSW_NB15_training-set.csv"
TEST_PATH  = "UNSW_NB15_testing-set.csv"

# 1) Read CSVs
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 2) Fill NaN in attack_cat
train_df = train_df.assign(attack_cat=train_df['attack_cat'].fillna('Normal'))
test_df = test_df.assign(attack_cat=test_df['attack_cat'].fillna('Normal'))

# 3) Encode 'attack_cat' as the label
attack_cat_encoder = LabelEncoder()
train_df['attack_cat'] = attack_cat_encoder.fit_transform(train_df['attack_cat'])
test_df['attack_cat'] = attack_cat_encoder.transform(test_df['attack_cat'])

# 4) Handle categorical columns more robustly
cat_cols = ['proto', 'service', 'state']
for col in cat_cols:
    # Get unique values from both train and test
    combined_categories = pd.concat([train_df[col], test_df[col]]).unique()
    le = LabelEncoder()
    le.fit(combined_categories.astype(str))
    
    # Transform separately
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

# Drop unused cols
drop_cols = ['id', 'label']
for c in drop_cols:
    if c in train_df.columns:
        train_df.drop(columns=c, inplace=True)
    if c in test_df.columns:
        test_df.drop(columns=c, inplace=True)

feature_cols = [c for c in train_df.columns if c != 'attack_cat']
X_train = train_df[feature_cols].values
y_train = train_df['attack_cat'].values
X_test = test_df[feature_cols].values
y_test = test_df['attack_cat'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Train X shape:", X_train.shape, "y shape:", y_train.shape)
print("Test  X shape:", X_test.shape,  "y shape:", y_test.shape)


Train X shape: (175341, 42) y shape: (175341,)
Test  X shape: (82332, 42) y shape: (82332,)


In [5]:
# # Instead of KNN, let's try to connect flows (rows) if they share the same (proto, service, state).
# NOT SURE IF IT WILL WORK!

# Let's do this for TRAIN. Each row is a node.
# # The columns for proto, service, state in the *scaled* matrix are no longer 
# # integers, so let's get them from the unscaled dataframe. That means we do:
# train_proto  = train_df['proto'].values
# train_service= train_df['service'].values
# train_state  = train_df['state'].values

# # Group rows by (proto, service, state)
# from collections import defaultdict
# groups = defaultdict(list)
# for i, (p, s, st) in enumerate(zip(train_proto, train_service, train_state)):
#     groups[(p,s,st)].append(i)

# edge_source = []
# edge_target = []
# # For each group, connect all nodes in that group fully
# # But if the group is huge, try to maybe limit the number of edges
# for triple, indices in groups.items():
#     if len(indices) > 1:
#         # connect all pairs in 'indices'
#         # for large groups, you might want to do a random subset 
#         # or skip a big group to avoid excessive memory usage
#         for i in range(len(indices) - 1):
#             for j in range(i+1, len(indices)):
#                 node_i = indices[i]
#                 node_j = indices[j]
#                 edge_source.append(node_i)
#                 edge_target.append(node_j)
#                 edge_source.append(node_j)
#                 edge_target.append(node_i)

# edge_index_train = np.vstack((edge_source, edge_target))
# edge_index_train = torch.tensor(edge_index_train, dtype=torch.long)

# print("edge_index_train shape:", edge_index_train.shape)

# Code above did not work well, maybe LOTS OF ADJACENCY

# To reduce the combinatorial explosion we set thresholds.
VERBOSE = True
MAX_NODES_PER_GROUP = 500   # if a group has more than this, sample nodes
MAX_EDGES_PER_GROUP = 10000 # if full connectivity would produce too many edges, sample pairs

def create_edge_index(proto, service, state, max_nodes=MAX_NODES_PER_GROUP, 
                      max_edges=MAX_EDGES_PER_GROUP, verbose=False):
    groups = defaultdict(list)
    for i, (p, s, st) in enumerate(zip(proto, service, state)):
        groups[(p, s, st)].append(i)
    
    if verbose:
        num_groups = len(groups)
        max_group_size = max(len(idxs) for idxs in groups.values())
        print(f"Total groups: {num_groups}, Maximum group size: {max_group_size}")
    
    edge_source = []
    edge_target = []
    
    for triple, indices in groups.items():
        group_size = len(indices)
        if group_size <= 1:
            continue  # no edge possible
        
        # If the group is huge (YES!!!), sample a subset of nodes first
        if group_size > max_nodes:
            if verbose:
                print(f"Group {triple} has {group_size} nodes; sampling {max_nodes} nodes.")
            indices = random.sample(indices, max_nodes)
            group_size = max_nodes

        # Compute total possible undirected edges (each added in both directions)
        total_edges = group_size * (group_size - 1) // 2
        if total_edges > max_edges:
            # sample a fixed number of edges
            if verbose:
                print(f"Group {triple} (size {group_size}) would produce {total_edges} edges; sampling {max_edges} pairs.")
            all_pairs = list(itertools.combinations(indices, 2))
            sampled_pairs = random.sample(all_pairs, max_edges)
            for i, j in sampled_pairs:
                edge_source.extend([i, j])
                edge_target.extend([j, i])
        else:
            # Full connectivity for the group
            for i in range(group_size - 1):
                for j in range(i + 1, group_size):
                    edge_source.extend([indices[i], indices[j]])
                    edge_target.extend([indices[j], indices[i]])
    # Stack into array: shape (2, num_edges)
    return np.vstack((edge_source, edge_target))


In [9]:
# For training data

import random
import itertools

train_proto   = train_df['proto'].values
train_service = train_df['service'].values
train_state   = train_df['state'].values

edge_index_train_np = create_edge_index(train_proto, train_service, train_state, 
                                          max_nodes=MAX_NODES_PER_GROUP, 
                                          max_edges=MAX_EDGES_PER_GROUP, verbose=VERBOSE)
edge_index_train = torch.tensor(edge_index_train_np, dtype=torch.long)
print("edge_index_train shape:", edge_index_train.shape)

Total groups: 169, Maximum group size: 44185
Group (113, 0, 4) has 44185 nodes; sampling 500 nodes.
Group (113, 0, 4) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 3, 4) has 3419 nodes; sampling 500 nodes.
Group (113, 3, 4) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 9, 4) has 5008 nodes; sampling 500 nodes.
Group (113, 9, 4) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 5, 4) has 18702 nodes; sampling 500 nodes.
Group (113, 5, 4) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (119, 0, 5) has 12043 nodes; sampling 500 nodes.
Group (119, 0, 5) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (6, 0, 5) has 2769 nodes; sampling 500 nodes.
Group (6, 0, 5) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (78, 0, 5) has 1770 nodes; sampling 500 nodes.
Group (78, 0, 5) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 4, 4) has 3993 

In [10]:
# Build the PyG Data for training
x_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

train_data = Data(
    x=x_train_tensor,
    y=y_train_tensor,
    edge_index=edge_index_train
)
print(train_data)
print("Train data: #nodes =", train_data.num_nodes, 
      "#edges =", train_data.num_edges, 
      "num_features =", train_data.num_features)

Data(x=[175341, 42], edge_index=[2, 1751590], y=[175341])
Train data: #nodes = 175341 #edges = 1751590 num_features = 42


In [13]:
# --- For test data ---
test_proto   = test_df['proto'].values
test_service = test_df['service'].values
test_state   = test_df['state'].values

edge_index_test_np = create_edge_index(test_proto, test_service, test_state, 
                                         max_nodes=MAX_NODES_PER_GROUP, 
                                         max_edges=MAX_EDGES_PER_GROUP, verbose=VERBOSE)
edge_index_test = torch.tensor(edge_index_test_np, dtype=torch.long)
x_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
test_data = Data(
    x=x_test_tensor,
    y=y_test_tensor,
    edge_index=edge_index_test
)
print(test_data)
print("Test data: #nodes =", test_data.num_nodes, 
      "#edges =", test_data.num_edges, 
      "num_features =", test_data.num_features)

Total groups: 165, Maximum group size: 25629
Group (119, 0, 5) has 6362 nodes; sampling 500 nodes.
Group (119, 0, 5) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (6, 0, 5) has 926 nodes; sampling 500 nodes.
Group (6, 0, 5) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 0, 4) has 25629 nodes; sampling 500 nodes.
Group (113, 0, 4) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 5, 4) has 8270 nodes; sampling 500 nodes.
Group (113, 5, 4) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (113, 0, 7) has 1655 nodes; sampling 500 nodes.
Group (113, 0, 7) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (78, 0, 5) has 524 nodes; sampling 500 nodes.
Group (78, 0, 5) (size 500) would produce 124750 edges; sampling 10000 pairs.
Group (97, 0, 5) (size 205) would produce 20910 edges; sampling 10000 pairs.
Group (113, 3, 4) has 1541 nodes; sampling 500 nodes.
Group (113, 3, 4) (size 500) woul

In [14]:
# GCN model
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_classes)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return x


In [15]:
# Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

num_features = train_data.num_features
num_classes = len(np.unique(y_train))
hidden_channels = 64

model = GCN(num_features, hidden_channels, num_classes).to(device)
train_data = train_data.to(device)
test_data = test_data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

Using device: cuda


In [18]:
# Actually train 
import time

def train_one_epoch():
    model.train()
    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index)
    loss = F.cross_entropy(out, train_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

epochs = 100
for epoch in range(1, epochs + 1):
    start_time = time.time()
    loss_val = train_one_epoch()
    elapsed = time.time() - start_time
    print(f"Epoch {epoch:02d}, Loss = {loss_val:.4f}, Time = {elapsed:.2f} sec")


Epoch 01, Loss = 0.8727, Time = 0.30 sec
Epoch 02, Loss = 0.8530, Time = 0.06 sec
Epoch 03, Loss = 0.8347, Time = 0.06 sec
Epoch 04, Loss = 0.8186, Time = 0.06 sec
Epoch 05, Loss = 0.8042, Time = 0.05 sec
Epoch 06, Loss = 0.7913, Time = 0.05 sec
Epoch 07, Loss = 0.7809, Time = 0.05 sec
Epoch 08, Loss = 0.7726, Time = 0.05 sec
Epoch 09, Loss = 0.7660, Time = 0.05 sec
Epoch 10, Loss = 0.7595, Time = 0.05 sec
Epoch 11, Loss = 0.7522, Time = 0.06 sec
Epoch 12, Loss = 0.7443, Time = 0.05 sec
Epoch 13, Loss = 0.7371, Time = 0.05 sec
Epoch 14, Loss = 0.7313, Time = 0.05 sec
Epoch 15, Loss = 0.7265, Time = 0.05 sec
Epoch 16, Loss = 0.7219, Time = 0.05 sec
Epoch 17, Loss = 0.7177, Time = 0.06 sec
Epoch 18, Loss = 0.7141, Time = 0.05 sec
Epoch 19, Loss = 0.7108, Time = 0.05 sec
Epoch 20, Loss = 0.7075, Time = 0.06 sec
Epoch 21, Loss = 0.7042, Time = 0.05 sec
Epoch 22, Loss = 0.7005, Time = 0.05 sec
Epoch 23, Loss = 0.6966, Time = 0.05 sec
Epoch 24, Loss = 0.6930, Time = 0.05 sec
Epoch 25, Loss =

In [21]:
# Results
def evaluate(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1).cpu().numpy()
        labels = data.y.cpu().numpy()
    return pred, labels

# Evaluate on TRAIN data
train_pred, train_labels = evaluate(train_data)
cm_train = confusion_matrix(train_labels, train_pred)
print("\n=== Confusion Matrix (Train) ===")
print(cm_train)
print("\n=== Classification Report (Train) ===")
print(classification_report(train_labels, train_pred, target_names=attack_cat_encoder.classes_))

# Evaluate on TEST data
test_pred, test_labels = evaluate(test_data)
cm_test = confusion_matrix(test_labels, test_pred)
print("\n=== Confusion Matrix (Test) ===")
print(cm_test)
print("\n=== Classification Report (Test) ===")
print(classification_report(test_labels, test_pred, target_names=attack_cat_encoder.classes_))



=== Confusion Matrix (Train) ===
[[  223     0     0  1559     1     0   172    45     0     0]
 [    0     0     0  1553    47     0    13   133     0     0]
 [    3     0    13 11352   288     3    72   527     6     0]
 [   40     0     5 30627  1349    15   377   978     2     0]
 [   14     0     4  2236 12849    18  2024  1036     3     0]
 [    4     0     0   692   140 39090    14    59     1     0]
 [  124     0     0  1278  6325     2 47858   413     0     0]
 [    4     0     0  3085   638     1   185  6578     0     0]
 [    0     0     0   113   318     0    32   660    10     0]
 [    2     0     0   109    17     0     0     2     0     0]]

=== Classification Report (Train) ===
                precision    recall  f1-score   support

      Analysis       0.54      0.11      0.18      2000
      Backdoor       0.00      0.00      0.00      1746
           DoS       0.59      0.00      0.00     12264
      Exploits       0.58      0.92      0.71     33393
       Fuzzers 

C:\Users\91965\anaconda3\envs\infra-env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\91965\anaconda3\envs\infra-env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\91965\anaconda3\envs\infra-env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [22]:
# Save model
torch.save(model, "gnn_model_actual_beta.pt")

# Save weights
torch.save(model.state_dict(), "gnn_model_actual_beta_weights.pt")

print("\nModel and weights saved locally!")


Model and weights saved locally!
